In [29]:
import pickle
import networkx as nx

G = pickle.load(open("./output/thirsty-crow-prompt2_nx_graph.pkl", "rb"))
G

In [30]:
query = "how was the crow able to drink water?"

In [31]:
import networkx as nx
import ollama
import json

with open("./input/config.json", "r") as file:
    config = json.load(file)

model = config['model_name']

system_prompt = """You are a helpful assistant that extracts the root form of the given word in lowercase.
Return only the root word in response."""

def get_word_root(text):
    response = ollama.chat(model=model, messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}
        ])
    # print(f'response={response['message']['content']}')
    resp_content = response['message']['content']
    return resp_content

# --- Step 2: Retrieve relevant triples ---
def search_kg(G, query):
    print(f"no. of edges in KG: {G.number_of_edges()}")
    results = []
    for u, v, d in G.edges(data=True):
        relation = d.get('label', None) if d else None
        relation = d.get('relation', None) if relation is None else relation
        triple = (u, relation, v)
        print(f'triple: {triple}')
        # naive: check if query words appear in triple

        if any(get_word_root(word) in " ".join(map(str, triple)).lower() for word in query.split()):
            results.append(triple)
    return results

triples = search_kg(G, query)
print(f'number of relevant triples found: {len(triples)}, {triples}')

# --- Step 3: Send to Ollama model ---
context = f"""
You are given facts from a knowledge graph:

{triples}

Answer the user query based ONLY on these facts.
Answer in full sentence.
Query: {query}
"""

response = ollama.chat(model="gemma3:4b-it-qat", messages=[{"role": "user", "content": context}])

print(response["message"]["content"])


no. of edges in KG: 9
triple: ('crow', 'fly_in_search_of', 'water')
triple: ('crow', 'see', 'water')
triple: ('crow', 'try_to_reach', 'water')
triple: ('crow', 'drink', 'water')
triple: ('crow', 'fly_away', 'water')
triple: ('crow', 'look_inside', 'pitcher')
triple: ('crow', 'drop_in', 'stone')
triple: ('pitcher', 'rise', 'water')
triple: ('stone', 'drop', 'pitcher')
number of relevant triples found: 8, [('crow', 'fly_in_search_of', 'water'), ('crow', 'see', 'water'), ('crow', 'try_to_reach', 'water'), ('crow', 'drink', 'water'), ('crow', 'fly_away', 'water'), ('crow', 'look_inside', 'pitcher'), ('crow', 'drop_in', 'stone'), ('stone', 'drop', 'pitcher')]
The crow was able to drink water by trying to reach it and then drinking from it.
